Loading a pre-trained BERT model

In [1]:
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import torch
import pandas as pd
import numpy as np
from Model import Model

ModuleNotFoundError: No module named 'transformers'

In [6]:
model_name = "howey/electra-base-mnli"
model = Model(model_name, num_labels=13)

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at howey/electra-base-mnli and are newly initialized because the shapes did not match:
- classifier.out_proj.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([13, 768]) in the model instantiated
- classifier.out_proj.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([13]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
train_data = pd.read_csv('data/edu_train.csv')[['source_article', 'updated_label']]
dev_data = pd.read_csv('data/edu_dev.csv')[['source_article', 'updated_label']]
test_data = pd.read_csv('data/edu_test.csv')[['source_article', 'updated_label']]

In [5]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    evaluation_strategy="steps",
    eval_steps=150,
    logging_dir='./logs',
    logging_steps=50,
    save_steps=150,
    save_total_limit=2,
    load_best_model_at_end=True
)

def compute_metrics(pred):
    predictions, labels = pred.predictions, pred.label_ids
    predictions = np.argmax(predictions, axis=1)
    
    accuracy = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='micro')
    precision = precision_score(labels, predictions, average='macro')
    recall = recall_score(labels, predictions, average='macro')

    return {
        'accuracy': accuracy,
        'f1': f1,
        'precision': precision,
        'recall': recall
        }

In [5]:
train_input_ids, train_attention_mask, train_labels_tensor = model.data_tensor(train_data)
dev_input_ids, dev_attention_mask, dev_labels_tensor = model.data_tensor(train_data)
test_input_ids, test_attention_mask, test_labels_tensor = model.data_tensor(test_data)

/root/lucaM/logical-fallacy-LLMs/Model.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(encoded.input_ids), torch.tensor(encoded.attention_mask), torch.tensor(self.label_encoder(data))


2
2
2


/root/lucaM/logical-fallacy-LLMs/Model.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(encoded.input_ids), torch.tensor(encoded.attention_mask), torch.tensor(self.label_encoder(data))
/root/lucaM/logical-fallacy-LLMs/Model.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(encoded.input_ids), torch.tensor(encoded.attention_mask), torch.tensor(self.label_encoder(data))


In [7]:
trainer = Trainer(
    model=model.get_model(),
    args=training_args,
    compute_metrics=compute_metrics,
    data_collator=lambda data: {
        'input_ids': torch.stack([item[0] for item in data]),
        'attention_mask': torch.stack([item[1] for item in data]),
        'labels': torch.stack([item[2] for item in data])
    },
    train_dataset=torch.utils.data.TensorDataset(train_input_ids, train_attention_mask, train_labels_tensor),
    eval_dataset=torch.utils.data.TensorDataset(dev_input_ids, dev_attention_mask, dev_labels_tensor),

    )    

In [8]:
# Fine-tune the model on your training data
trainer.train()

/root/miniconda3/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
150,1.561800,1.230011,0.621417,0.621417,0.503575,0.500337


/root/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/root/miniconda3/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=232, training_loss=1.6686043246039028, metrics={'train_runtime': 96.2353, 'train_samples_per_second': 76.853, 'train_steps_per_second': 2.411, 'total_flos': 904661036844432.0, 'train_loss': 1.6686043246039028, 'epoch': 4.0})

In [9]:
# Evaluate the model on the test set
results = trainer.evaluate(eval_dataset=torch.utils.data.TensorDataset(test_input_ids, test_attention_mask, test_labels_tensor))

/root/miniconda3/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [10]:
print(results)

{'eval_loss': 1.747412919998169, 'eval_accuracy': 0.49, 'eval_f1': 0.49, 'eval_precision': 0.29284684645063713, 'eval_recall': 0.3779734737635573, 'eval_runtime': 1.5701, 'eval_samples_per_second': 191.072, 'eval_steps_per_second': 6.369, 'epoch': 4.0}


In [11]:
from performance import PerformanceSaver
PerformanceSaver().save_performance(model_name=model_name, results=results)

In [12]:
del model
del trainer
torch.cuda.empty_cache()